In [ ]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 KB 5.0 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt
import sklearn
from sklearn.model_selection import train_test_split

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()

170498071/170498071 [==============================] - 2s 0us/step


In [ ]:
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

In [ ]:
# split training into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
def BasicCNN():
  model = keras.Sequential()
  model.add(keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation="relu", input_shape=(32, 32, 3)))
  model.add(keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation="relu"))
  model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))

  model.add(keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation="relu", padding="same"))
  model.add(keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation="relu"))
  model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))

  model.add(keras.layers.Flatten())

  model.add(keras.layers.Dense(units=512, activation="relu"))
  model.add(keras.layers.Dense(units=10, activation="softmax"))

  opt = keras.optimizers.Adam(learning_rate=0.01)
  model.compile(optimizer=opt, loss="sparse_categorical_crossentropy", metrics=['accuracy'])

  return model

In [ ]:
model = BasicCNN()

In [ ]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20)

Epoch 1/20
1250/1250 [==============================] - 8s 5ms/step - loss: 2.0044 - accuracy: 0.2714 - val_loss: 1.7237 - val_accuracy: 0.3921
Epoch 2/20
1250/1250 [==============================] - 7s 5ms/step - loss: 1.6246 - accuracy: 0.4178 - val_loss: 1.5953 - val_accuracy: 0.4371
Epoch 3/20
1250/1250 [==============================] - 6s 5ms/step - loss: 1.4343 - accuracy: 0.4851 - val_loss: 1.3435 - val_accuracy: 0.5135
Epoch 4/20
1250/1250 [==============================] - 7s 5ms/step - loss: 1.2986 - accuracy: 0.5371 - val_loss: 1.2377 - val_accuracy: 0.5577
Epoch 5/20
1250/1250 [==============================] - 8s 6ms/step - loss: 1.1868 - accuracy: 0.5837 - val_loss: 1.1969 - val_accuracy: 0.5764
Epoch 6/20
1250/1250 [==============================] - 6s 5ms/step - loss: 1.0885 - accuracy: 0.6194 - val_loss: 1.1000 - val_accuracy: 0.6135
Epoch 7/20
1250/1250 [==============================] - 7s 5ms/step - loss: 0.9958 - accuracy: 0.6524 - val_loss: 1.1038 - val_accuracy:

In [ ]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 12s 39ms/step - loss: 2.3732 - accuracy: 0.6480


[2.373150110244751, 0.6480000019073486]

In [ ]:
class MyHyperModel(kt.HyperModel):
  def build(self, hp):
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(
        filters=hp.Int("conv_1_layers", 32, 96, step=32),
        kernel_size=(3,3),
        activation="relu", 
        input_shape=(32, 32, 3)
    ))

    model.add(keras.layers.Conv2D(
        filters=hp.Int("conv_2_layers", 32, 96, step=32),
        kernel_size=(3,3),
        activation="relu"
    ))

    model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))

    model.add(keras.layers.Dropout(hp.Float('dropout_2', min_value=0, max_value=1, step=0.2)))

    model.add(keras.layers.Conv2D(
        filters=64, 
        kernel_size=(3,3), 
        activation="relu",
        padding="same",
    ))

    model.add(keras.layers.Conv2D(
        filters=64, 
        kernel_size=(3,3), 
        activation="relu",
    ))

    model.add(keras.layers.Dropout(hp.Float('dropout_3', min_value=0, max_value=1, step=0.2)))

    model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))

    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(units=hp.Int("dense_1", 64, 512, step=32), activation="relu"))
    model.add(keras.layers.Dense(units=10, activation="softmax"))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=keras.losses.SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])

    return model
  
  def fit(self, hp, model, *args, **kwargs):
    return model.fit(
        *args,
        batch_size=hp.Int("batch_size", 32, 256, step=32),
        **kwargs,
    )


In [ ]:
tuner = kt.Hyperband(MyHyperModel(), objective='val_accuracy', overwrite=True)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
tuner.search(X_train, y_train, epochs=20, validation_data=(X_val, y_val), callbacks=[stop_early])

Trial 254 Complete [00h 01m 25s]
val_accuracy: 0.10537499934434891

Best val_accuracy So Far: 0.7802500128746033
Total elapsed time: 02h 44m 49s


In [ ]:
best_model = tuner.get_best_models()[0]

In [ ]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0245 summary
Hyperparameters:
conv_1_layers: 32
conv_2_layers: 96
dropout_2: 0.4
dropout_3: 0.8
dense_1: 96
learning_rate: 0.001
batch_size: 256
tuner/epochs: 100
tuner/initial_epoch: 34
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0238
Score: 0.7864999771118164

Trial 0238 summary
Hyperparameters:
conv_1_layers: 32
conv_2_layers: 96
dropout_2: 0.4
dropout_3: 0.8
dense_1: 96
learning_rate: 0.001
batch_size: 256
tuner/epochs: 34
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.7796249985694885

Trial 0143 summary
Hyperparameters:
conv_1_layers: 32
conv_2_layers: 64
dropout_2: 0.4
dropout_3: 0.8
dense_1: 384
learning_rate: 0.001
batch_size: 96
tuner/epochs: 34
tuner/initial_epoch: 12
tuner/bracket: 4
tuner/round: 3
tuner/trial_id: 0139
Score: 0.7735000252723694

Trial 0146 summary
Hyperparameters:
conv_1_layers: 32
conv_2_layers: 64
dropout_2: 0.4


In [ ]:
best_model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 4ms/step - loss: 0.6518 - accuracy: 0.7896


[0.6518431901931763, 0.7896000146865845]